In [1]:
import pandas as pd
import os

save_path = os.path.join(os.getcwd(), 'dataset_merge')
merged_df = pd.read_pickle(os.path.join(save_path, 'dataset_merge.pkl'))

In [2]:
merged_df.head()

,DateTime,Most_Recent_Forecast_MW,Day_Ahead_Forecast_MW,Week_Ahead_Forecast_MW,Real_Time_Measurement_MW,Corrected_Measurement_MW,Monitored_Capacity_MWp,Day_Ahead_11h00_MW
0,2012-02-01 00:00:00,0.0,0.0,0.0,0.0,0.0,400.02,0.0
1,2012-02-01 00:15:00,0.0,0.0,0.0,0.0,0.0,400.02,0.0
2,2012-02-01 00:30:00,0.0,0.0,0.0,0.0,0.0,400.02,0.0
3,2012-02-01 00:45:00,0.0,0.0,0.0,0.0,0.0,400.02,0.0
4,2012-02-01 01:00:00,0.0,0.0,0.0,0.0,0.0,400.02,0.0


In [3]:
# create new pandas dataframe
# columns 1: Time 05:00:00 to 
# columns 2: Season (Spring, Summer, Autumn, Winter)
# columns 3: Day     (1,2,3, ... 31) (28, 29, 30, 31 for some months)
# columns 4: Real_Time_Measurement_MW
# columns 5: Forecasted_Real_Time_Measurement_MW ( selec next day Real_Time_Measurement_MW to be forecasted)

Season = ['Spring', 'Summer', 'Autumn', 'Winter']
# condition for each season
merged_df.reset_index(inplace=True)
merged_df['DateTime'] = pd.to_datetime(merged_df['DateTime'])
merged_df.set_index('DateTime', inplace=True)

Spring = (merged_df.index.month >= 3) & (merged_df.index.month <= 5)
Summer = (merged_df.index.month >= 6) & (merged_df.index.month <= 8)
Autumn = (merged_df.index.month >= 9) & (merged_df.index.month <= 11)
Winter = (merged_df.index.month == 12) | (merged_df.index.month <= 2)

# create new dataframe
new_df = pd.DataFrame(columns=['Time', 'Season', 'Day', 'Real_Time_Measurement_MW', 'Forecast'])

rows = []
for i in range(merged_df.shape[0]):
    row = None
    if merged_df.index[i].time() >= pd.to_datetime('05:00:00').time() and merged_df.index[i].time() <= pd.to_datetime('22:00:00').time():
        row = {'Time': merged_df.index[i],
               'Season': Season[0] if Spring[i] else Season[1] if Summer[i] else Season[2] if Autumn[i] else Season[3],
               'Day': merged_df.index[i].day,
               'Real_Time_Measurement_MW': merged_df['Real_Time_Measurement_MW'][i],
               'Forecast': merged_df['Real_Time_Measurement_MW'][i+96] if i < merged_df.shape[0] - 96 else None}
    if row:
        rows.append(row)

new_df = pd.concat([new_df, pd.DataFrame(rows)], ignore_index=True)

C:\Users\krits\AppData\Local\Temp\ipykernel_8536\3052154582.py:29: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  'Real_Time_Measurement_MW': merged_df['Real_Time_Measurement_MW'][i],
C:\Users\krits\AppData\Local\Temp\ipykernel_8536\3052154582.py:30: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  'Forecast': merged_df['Real_Time_Measurement_MW'][i+96] if i < merged_df.shape[0] - 96 else None}
C:\Users\krits\AppData\Local\Temp\ipykernel_8536\3052154582.py:34: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA

In [4]:
# save new dataframe to pickle and xlsx
save_path = os.path.join(os.getcwd(), 'dataset_season')

new_df.to_pickle(os.path.join(save_path, 'Season_df.pkl'))
new_df.to_excel(os.path.join(save_path, 'Season_df.xlsx'))

In [5]:
new_df.head()

,Time,Season,Day,Real_Time_Measurement_MW,Forecast
0,2012-02-01 05:00:00,Winter,1,0.0,0.0
1,2012-02-01 05:15:00,Winter,1,0.0,0.0
2,2012-02-01 05:30:00,Winter,1,0.0,0.0
3,2012-02-01 05:45:00,Winter,1,0.0,0.0
4,2012-02-01 06:00:00,Winter,1,0.0,0.0
